<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/AgeGenderModelLoadingFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def load_age_gender_models():
    """Download and load the pre-trained age and gender models using direct URLs"""
    import os

    # Define more reliable direct URLs for the model files
    age_proto_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
    age_model_url = "https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/age_net.caffemodel"
    gender_proto_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
    gender_model_url = "https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/gender_net.caffemodel"

    # Define filenames
    age_proto_file = "age_deploy.prototxt"
    age_model_file = "age_net.caffemodel"
    gender_proto_file = "gender_deploy.prototxt"
    gender_model_file = "gender_net.caffemodel"

    # Use wget for more reliable downloads (with progress indicators)
    print("Downloading model files...")
    !wget -O {age_proto_file} {age_proto_url} --no-check-certificate
    !wget -O {age_model_file} {age_model_url} --no-check-certificate
    !wget -O {gender_proto_file} {gender_proto_url} --no-check-certificate
    !wget -O {gender_model_file} {gender_model_url} --no-check-certificate

    # Verify the files exist and have contents
    for file in [age_proto_file, age_model_file, gender_proto_file, gender_model_file]:
        if not os.path.exists(file) or os.path.getsize(file) == 0:
            raise FileNotFoundError(f"Failed to download or empty file: {file}")
        print(f"Successfully downloaded {file} ({os.path.getsize(file)} bytes)")

    # Load the models
    print("Loading models into OpenCV...")
    age_net = cv2.dnn.readNet(age_model_file, age_proto_file)
    gender_net = cv2.dnn.readNet(gender_model_file, gender_proto_file)

    print("Age and gender models loaded successfully")
    return age_net, gender_net

### Step 1: Import Libraries and Set Up Environment


In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from roboflow import Roboflow
import os

# Handle encoding
os.environ["PYTHONIOENCODING"] = "utf-8"